In [1]:
#1) Import all necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import ee
import geemap
import geopandas as gpd


In [2]:
from constants import   STUDY_BOUNDARY_PATH


# Import Landsat8 OLI

https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#terms-of-use

 Landsat-8 image courtesy of the U.S. Geological Survey

In [3]:
#ee.Authenticate()
ee.Initialize()

In [4]:
# Create an interactive map
Map = geemap.Map()

# Display the map
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
# Define the collection
collection = "LANDSAT/LC08/C02/T1_L2"

In [ ]:
shapefile_path = STUDY_BOUNDARY_PATH
study_boundary = gpd.read_file(shapefile_path)

In [ ]:
# Load the study area
ee_boundary = geemap.geopandas_to_ee(study_boundary)


In [ ]:
aoi = ee_boundary.geometry()

In [ ]:
aoi

In [ ]:
# Define the dates of interest
dates = ['2021-07-06', '2021-07-22', '2021-08-07', '2021-10-10', '2021-10-26', '2021-11-11']


In [ ]:


# Loop through the dates and get the imagery
for date in dates:
    start_date = ee.Date(date)
    end_date = start_date.advance(1, 'day')

    # Filter the image collection by date and area
    image = ee.ImageCollection(collection) \
        .filterDate(start_date, end_date) \
        .filterBounds(aoi) \
        .first()  # get the first image that matches the filters

    if image:  # check if image exists
        print(f"Image for date {date} found!")
        vis_params = {
            'bands': ['SR_B4', 'SR_B3', 'SR_B2'],  # specify the bands for RGB
            'min': 0,
            'max': 3000,
            'gamma': 1.4
        }
        Map.addLayer(image, vis_params, date)  # add the image to the map
    else:
        print(f"No image found for date {date}")

# Set the map to focus on the study area
Map.centerObject(ee_boundary, zoom=10)



In [ ]:
export_params = {
    'scale': 30, # Resolution in meters
    'region': ee_boundary, # Export only the region of interest
    'crs': 'EPSG:4326', # Coordinate reference system (optional)
    'fileFormat': 'GeoTIFF', # Export format (GeoTIFF or other supported formats)
    'fileNamePrefix': 'clipped_hansen_dataset', # Prefix for the exported file name
}
# Get the geometry and scale (resolution) of the clipped raster
geometry = ee_boundary.geometry()
scale = export_params['scale']

# Compute the pixel dimensions of the exported raster
dimensions = ee.Image.pixelLonLat().reproject(geometry.projection().atScale(scale)).reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=geometry,
    scale=scale,
    maxPixels=1e10
).getInfo()

pixel_width = dimensions['longitude_max'] - dimensions['longitude_min']
pixel_height = dimensions['latitude_max'] - dimensions['latitude_min']

# Estimate the number of pixels
num_pixels = pixel_width * pixel_height
print(f"Number of pixels: {num_pixels}")

# Estimate the file size in bytes (assuming 4 bytes per pixel per band for GeoTIFF format)``
file_size_bytes = num_pixels * num_bands * 4
print(f"Estimated file size (bytes): {file_size_bytes}")





# Export Deforestation and TreeCover Files



In [ ]:
# Extract the first feature from the FeatureCollection
ee_boundary_feature = ee_boundary.first()

# Get the geometry of the feature
ee_boundary_geometry = ee_boundary_feature.geometry()

# Get the coordinates of the geometry
ee_boundary_coordinates = ee_boundary_geometry.coordinates().getInfo()




In [ ]:
# Update the export_params dictionary
export_params = {
    'scale': 30,
    'region': ee_boundary_coordinates,
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'fileNamePrefix': 'clipped_hansen_lossyear',
}

# Your existing export task code
export_task = ee.batch.Export.image.toDrive(
    image=clipped_hansen_dataset.select('lossyear'),
    description='lossyear',
    folder='hansen_data', # specify a folder in your Google Drive
    maxPixels=1e10,
    **export_params
)

export_task.start()

A new empty folder will appear in the google drive associated with Earth Engine account.
The export process takes a considerable amount of time, progress can be monitored at
https://code.earthengine.google.com/tasks